In [ ]:
!pip install --upgrade openai>/dev/null #make sure to use the latest version of the library

In [ ]:

import time
import openai
import os
import pandas as pd
import torch
openai.api_key = ""
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

In [ ]:
class EnglishSudaneseDataset(torch.utils.data.Dataset):
  def __init__(self, df, tokenizer, max_len=16):
    self.max_length = max_len
    self.df = df
    self.df = self.df[~self.df["Sudanese"].isna()].reset_index()
    self.tokenizer  = tokenizer
  def __len__(self):
    return self.df.shape[0]-1

  def __getitem__(self, idx):

    src = self.df.loc[idx, "English"]
    tgt = self.df.loc[idx, "Sudanese"]
    model_inputs = self.tokenizer(src, text_target=tgt, truncation=True, max_length=self.max_length,padding="max_length",return_tensors="pt")
    return {"input_ids":model_inputs["input_ids"][0,:],"labels" :model_inputs['labels'][0,:]}

In [ ]:
def compute_corpus_bleu(references, predictions, ngrams):
  references = [[sentence.split()] for sentence in references]
  predictions = [sentence.split() for sentence in predictions]
  weights = [1/ngrams for i in range(1,ngrams+1)] + [0 for i in range(4-ngrams)]
  print (weights)
  return corpus_bleu(references, predictions, weights=weights)



# Experiment 1:

#### Translate Sudanese-English using ChatGPT

#### Load Data

In [ ]:
flores_eng_sd = pd.read_excel("/content/eng_Latn_devtest_translated_0_400_401_500_601_699_701_800_801_1011_final.xlsx")

In [ ]:
# filter out nulls
flores_eng_sd = pd.read_excel("/content/eng_Latn_devtest_translated_0_400_401_500_601_699_701_800_801_1011_final.xlsx")
flores_eng_sd = flores_eng_sd[~flores_eng_sd["Sudanese"].isna()]

In [ ]:
sudanese = flores_eng_sd.loc[:, "Sudanese"].tolist()

In [ ]:
english = flores_eng_sd.loc[:, "English"].tolist()

In [ ]:
def translate(sentences):
  sentences = [s.strip().replace("\n", "") for s in sentences]
  str_sentences = '\n'.join(sentences)
  prompt = f"""
  follow these instructions strictly.
  - translate these ten sentences from sudanese to english.
  - the sentences are separated by new line.
  - only output the translation.
  - output a new line after each sentence translation.
  {str_sentences}
  """
  #print (prompt)
  completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[{"role": "user", "content": prompt}])
#print(completion)
  time.sleep(20)
  result= (completion["choices"][0]["message"]["content"])
  return result.split("\n")


In [ ]:
result = translate(sudanese[-10:])
assert len(result) == 10

In [ ]:
result

In [ ]:
a = [12]
a.extend([1,2,3])
a

[12, 1, 2, 3]

In [ ]:
translations = []

In [ ]:
for i in range(0,len(sudanese), 10):
  print (f"translating {i} - {i+10}")
  result = translate(sudanese[i:i+10])
  while len(result)!= len(sudanese[i:i+10]):
    print ("Length is not equal , trying to translate again!")
    result =  translate(sudanese[i:i+10])
  translations.extend(result)

In [ ]:
translations.append("In many instances, workers are supposed to consult their supervisors before making any decision and it is expected of them to listen to the president's words without any questions.")

In [ ]:
(sudanese[-1])

'  في مرات كتيرة العمال مفروض يسألو رؤساءهم قبل ما يتخذو اي قرار و متوقع منهم يسمعو كلام الرئيس بدون اي سؤال.'

In [ ]:
results = translate(sudanese[140:150])
len(results)

17

In [ ]:
df = pd.DataFrame({"Sudanese":pd.Series(sudanese), "English(GT)":pd.Series(flores_eng_sd.loc[:, "English"].tolist()),"English(Predicted)": pd.Series(translations) })

In [ ]:
df.tail()

In [ ]:
[compute_corpus_bleu(df["English(GT)"].tolist(), df["English(Predicted)"].tolist(), i) for i in range(1,5)]

[1.0, 0, 0, 0]
[0.5, 0.5, 0, 0]
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0]
[0.25, 0.25, 0.25, 0.25]


[0.5783086438114592,
 0.44543009020914487,
 0.3526018615151522,
 0.28400751342188985]

In [ ]:
df.to_excel("flores_sd_eng_chatgpt_911_911.xlsx")

In [ ]:
results = [i for i in results if i!=""]

In [ ]:
translations.extend(results)

In [ ]:
translations

#### Measure BLEU Score


# Experiment 2:

Translate Flores English-Sudanese

In [ ]:
def translate(sentences):
  sentences = [s.strip().replace("\n", "") for s in sentences]
  str_sentences = '\n'.join(sentences)
  prompt = f"""
  follow these instructions strictly.
  - make the translations as close as possible to sudanese dialect.
  - translate these ten sentences from english to sudanese dialect.
  - the sentences are separated by new line.
  - only output the translation and do not output anything else.
  - output a new line after each sentence translation.
  {str_sentences}
  """
  #print (prompt)
  completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[{"role": "user", "content": prompt}])
#print(completion)
  time.sleep(20)
  result= (completion["choices"][0]["message"]["content"])
  return result.split("\n")


In [ ]:
translations = []

In [ ]:
sudanese = flores_eng_sd.loc[:, "Sudanese"].tolist()
english = flores_eng_sd.loc[:, "English"].tolist()

In [ ]:
for i in range(0,len(sudanese), 10):
  print (f"translating {i} - {i+10}")
  result = translate(english[i:i+10])
  while len(result)!= len(english[i:i+10]):
    print ("Length is not equal , trying to translate again!")
    result =  translate(english[i:i+10])
  translations.extend([(i+j,result[j]) for j in range(len(result))])

In [ ]:
translations

In [ ]:
english[15:20]

['USA Gymnastics supports an independent investigation that may shine light on how abuse of the proportion described so courageously by the survivors of Larry Nassar could have gone undetected for so long and embraces any necessary and appropriate changes.\n',
 'USA Gymnastics and the USOC have the same goal — making the sport of gymnastics, and others, as safe as possible for athletes to follow their dreams in a safe, positive and empowered environment.\n',
 'Throughout 1960s, Brzezinski worked for John F. Kennedy as his advisor and then the Lyndon B. Johnson administration.\n',
 'During the 1976 selections he advised Carter on foreign policy, then served as National Security Advisor (NSA) from 1977 to 1981, succeeding Henry Kissinger.\n',
 'As NSA, he assisted Carter in diplomatically handling world affairs, such as the Camp David Accords, 1978; normalizing US–China relations thought the late 1970s; the Iranian Revolution, which led to the Iran hostage crisis, 1979; and the Soviet in

In [ ]:
result = translate(english[15:20])

In [ ]:
result

['I cannot complete this task as "Sudanese dialect" is not a specific language. Sudan has over 400 dialects and around 140 languages, so it is necessary to specify which dialect to translate to.']

In [ ]:
result = translate()

# Experiment 3:
Translate English Arabic using ChatGPT

In [ ]:
def translate(sentences):
  sentences = [s.strip().replace("\n", "") for s in sentences]
  str_sentences = '\n'.join(sentences)
  prompt = f"""
  follow these instructions strictly.
  - translate these ten sentences from english to arabic.
  - the sentences are separated by new line.
  - only output the translation and do not output anything else.
  - output a new line after each sentence translation.
  {str_sentences}
  """
  alt_prompt = f"""
  follow these instructions strictly.
  - translate these five sentences from english to arabic.
  - the sentences are separated by new line.
  - only output the translation and do not output anything else.
  - output a new line after each sentence translation.
  {str_sentences}
  """
  #print (prompt)
  completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[{"role": "user", "content": prompt}])
#print(completion)
  time.sleep(20)
  result= (completion["choices"][0]["message"]["content"])
  return result.split("\n")


In [ ]:
flores_eng_ar = pd.read_excel("/content/eng_Latn_arb_Arab.devtest.xlsx")
flores_eng_sd = pd.read_excel("/content/eng_Latn_devtest_translated_0_400_401_500_601_699_701_800_801_1011_final.xlsx")

In [ ]:
# filter out nulls
flores_eng_ar = flores_eng_ar[~flores_eng_sd["Sudanese"].isna()]
flores_eng_ar

,Unnamed: 0,English,Arabic
0,0,"""We now have 4-month-old mice that are non-dia...","أضاف قائلاً، ""لدينا الآن فئران تبلغ من العمر 4..."
1,1,"Dr. Ehud Ur, professor of medicine at Dalhousi...",نبه الدكتور إيهود أور -أستاذ الطب في جامعة دال...
2,2,"Like some other experts, he is skeptical about...",مثل باقي الخبراء الآخرين، فهو متشكك فيما إذا ك...
3,3,"On Monday, Sara Danius, permanent secretary of...",في يوم الإثنين ، أعلنت سارة دانيوس -السكرتيرة ...
4,4,"Danius said, ""Right now we are doing nothing. ...","قال دانيوس ""لن نقوم الآن بفعل أي شيء. لقد قمت ..."
...,...,...,...
1007,1007,"As the areas are sparsely populated, and light...",بما أن المناطق مأهولة قليلاً والتلوث الضوئي عل...
1008,1008,Japanese work culture is more hierarchical and...,ثقافة العمل اليابانية أكثر هرمية ورسمية مما اع...
1009,1009,"Suits are standard business attire, and cowork...",البدلاتُ هي ملابسُ العملِ التقليديّة، وينادي ز...
1010,1010,"Workplace harmony is crucial, emphasizing grou...",و يُعد الانسجام في مكان العمل أمراَ بالغ الأهم...


In [ ]:
english = flores_eng_ar["English"].tolist()
arabic = flores_eng_ar["Arabic"].tolist()

In [ ]:
for senetence in english[740:750]:
  print (senetence)

Now widely available throughout the archipelago, Javanese cuisine features an array of simply seasoned dishes, the predominant flavorings the Javanese favor being peanuts, chillies, sugar (especially Javanese coconut sugar) and various aromatic spices.
Stirrups are supports for the rider's feet that hang down on either side of the saddle.
They provide greater stability for the rider but can have safety concerns due to the potential for a rider's feet to get stuck in them.
If a rider is thrown from a horse but has a foot caught in the stirrup, they could be dragged if the horse runs away. To minimize this risk, a number of safety precautions can be taken.
First, most riders wear riding boots with a heel and a smooth, quite narrow, sole.
Next, some saddles, particularly English saddles, have safety bars that allow a stirrup leather to fall off the saddle if pulled backwards by a falling rider.
Cochamó Valley - Chile's premier climbing destination, known as the Yosemite of South America, 

In [ ]:
tranlated_sentences_740_750 = [
    'الآن متاحة على نطاق واسع في جميع أنحاء الأرخبيل، تتميز المأكولات الجاوية بمجموعة من الأطباق المتبلة ببساطة، حيث تكون النكهات الرئيسية التي يفضلها الجاويون هي الفول السوداني والفلفل الحار والسكر (خاصة سكر جوز الهند الجاوي) ومجموعة متنوعة من البهارات العطرية.',
    'الأذرع هي أدوات دعم لأقدام الفارس تتدلى على جانبي السرج.',
    'توفر استقرارًا أكبر للفارس ولكنها يمكن أن تشكل مخاطر أمان بسبب احتمال علق قدم الفارس فيها.',
    'إذا تم رمي الفارس من الخيل ولكن قدمه ملتصقة في الأذرع، فقد يتم سحبه إذا هرب الخيل. للحد من هذا الخطر، يمكن اتخاذ عدد من الاحتياطات الأمان',
    'ولاً، يرتدي معظم الفرسان حذاء ركوب بكعب ونعل ناعم وضيق نوعًا ما.',
    'ثم، بعض السروج، وخاصة السروج الإنجليزية، تحتوي على أعمدة أمان تسمح لحزام الأذرع بالسقوط من السرج إذا تم سحبها للخلف بواسطة الفارس الساقط',
    'وادي كوتشامو - وجهة تسلق رئيسية في تشيلي، المعروفة باسم يوسمايت في أمريكا الجنوبية، بتنوعها من الجدران الصخرية الكبيرة والمحاجر',
    'تشتمل القمم على مناظر خلابة من القمم. يقوم المتسلقون من جميع أنحاء العالم بإنشاء مسارات جديدة باستمرار بين إمكانياتها اللانهائية للجدران',
    'الرياضات الثلجية الهابطة، والتي تشمل التزلج وركوب الثلج، هي رياضات شعبية تنطوي على الانزلاق على التضاريس المغطاة بالثلج باستخدام الزلاجات أو لوح التزلج المثبتة على القدمين',
    'التزلج هو نشاط سفر رئيسي يتمتع به العديد من المشجعين، ويعرف بأحيان بـ "سكي بامز"، حيث يخططون لقضاء عطلات كاملة حول التزلج في موقع معين.'

]

In [ ]:
tranlated_sentences_740_750 = translated_sentences.split('\n')

In [ ]:
len(tranlated_sentences_740_750)

10

In [ ]:
translated_sentences[10]

' '

In [ ]:
arabic

In [ ]:
translations_df = pd.read_excel('/content/translations_chatgpt_flores_eng_ar_0_910.xlsx')

In [ ]:
translations_df['translation']

0      (0, '"لدينا الآن فئران بعمر 4 أشهر غير مصابة ب...
1      (1, 'حذر الدكتور إيهود أور ، أستاذ الطب في جام...
2      (2, 'مثل بعض الخبراء الآخرين ، فهو متشكك من مد...
3      (3, 'في يوم الاثنين، أعلنت سارة دانيوس، الأمين...
4      (4, 'قالت دانيوس: "في الوقت الحالي لا نفعل شيئ...
                             ...                        
896    (906, 'نظرًا لأن المناطق قليلة السكان ، ولا يو...
897    (907, 'ثقافة العمل اليابانية هي أكثر تسلسلًا و...
898    (908, 'البدلات هي ملابس الأعمال القياسية ، ويد...
899    (909, 'الانسجام في مكان العمل أمر حاسم ، حيث ي...
900    (910, 'يجب على العمال غالبًا الحصول على موافقة...
Name: translation, Length: 901, dtype: object

In [ ]:
all_translations = [eval(s) for s in translations_df["translation"].tolist()]

In [ ]:
for idx, sentence in enumerate(tranlated_sentences_740_750):
  all_translations.append((idx+740, sentence))

In [ ]:
all_translations.sort(key=lambda x: x[0])

In [ ]:
all_translations = [s[1] for s in all_translations]

In [ ]:
[compute_corpus_bleu(arabic, all_translations,i ) for i in range(1,5)]

[1.0, 0, 0, 0]
[0.5, 0.5, 0, 0]
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0]
[0.25, 0.25, 0.25, 0.25]


[0.45074537365077355,
 0.3289574881704043,
 0.2472394974989266,
 0.18863569293946997]

In [ ]:
df = pd.DataFrame({"English":pd.Series(english), "Arabic(GT)":pd.Series(arabic),"Arabic(Predicted)": pd.Series(all_translations) })

In [ ]:
df


,English,Arabic(GT),Arabic(Predicted)
0,"""We now have 4-month-old mice that are non-dia...","أضاف قائلاً، ""لدينا الآن فئران تبلغ من العمر 4...","""لدينا الآن فئران بعمر 4 أشهر غير مصابة بالسكر..."
1,"Dr. Ehud Ur, professor of medicine at Dalhousi...",نبه الدكتور إيهود أور -أستاذ الطب في جامعة دال...,حذر الدكتور إيهود أور ، أستاذ الطب في جامعة دا...
2,"Like some other experts, he is skeptical about...",مثل باقي الخبراء الآخرين، فهو متشكك فيما إذا ك...,مثل بعض الخبراء الآخرين ، فهو متشكك من مدى إمك...
3,"On Monday, Sara Danius, permanent secretary of...",في يوم الإثنين ، أعلنت سارة دانيوس -السكرتيرة ...,في يوم الاثنين، أعلنت سارة دانيوس، الأمين الدا...
4,"Danius said, ""Right now we are doing nothing. ...","قال دانيوس ""لن نقوم الآن بفعل أي شيء. لقد قمت ...","قالت دانيوس: ""في الوقت الحالي لا نفعل شيئًا. ل..."
...,...,...,...
906,"As the areas are sparsely populated, and light...",بما أن المناطق مأهولة قليلاً والتلوث الضوئي عل...,نظرًا لأن المناطق قليلة السكان ، ولا يوجد بالت...
907,Japanese work culture is more hierarchical and...,ثقافة العمل اليابانية أكثر هرمية ورسمية مما اع...,ثقافة العمل اليابانية هي أكثر تسلسلًا ورسمية م...
908,"Suits are standard business attire, and cowork...",البدلاتُ هي ملابسُ العملِ التقليديّة، وينادي ز...,البدلات هي ملابس الأعمال القياسية ، ويدعو الزم...
909,"Workplace harmony is crucial, emphasizing grou...",و يُعد الانسجام في مكان العمل أمراَ بالغ الأهم...,الانسجام في مكان العمل أمر حاسم ، حيث يؤكد على...


In [ ]:
df.to_excel("flores_eng_ar_chatgpt_zeroshot_911_911.xlsx")

In [ ]:
#740 - 750

In [ ]:
for i in range(750,len(arabic), 10):
  print (f"translating {i} - {i+10}")
  result = translate(english[i:i+10])
  while len(result)!= len(english[i:i+10]):
    print ("Length is not equal , trying to translate again!")
    result =  translate(english[i:i+10])
  translations.extend([(i+j,result[j]) for j in range(len(result))])

translating 750 - 760
translating 760 - 770
translating 770 - 780
translating 780 - 790
translating 790 - 800
translating 800 - 810
translating 810 - 820
translating 820 - 830
translating 830 - 840
translating 840 - 850
translating 850 - 860
translating 860 - 870
translating 870 - 880
translating 880 - 890
translating 890 - 900
translating 900 - 910
Length is not equal , trying to translate again!
translating 910 - 920


In [ ]:
translations

In [ ]:
english[740:745]

In [ ]:
result = translate(english[740:745])

In [ ]:
len(result)

10

In [ ]:
result

['الآن متاحة على نطاق واسع في الأرخبيل، تتميز المأكولات الجاوية بمجموعة من الأطعمة المتبلة بسيطة، حيث تشتهر بنكهات المكسرات والفلفل الحار والسكر (خاصة سكر جوز الهند الجاوي) ومختلف التوابل العطرية.',
 'الأرجل عبارة عن دعامات لأقدام الفارس تتدلى على كل جانب من السرج.',
 'توفر الأرجل مزيدًا من الاستقرار للفارس ولكنها قد تسبب مخاوف أمنية بسبب احتمال أن يلتصق قدم الفارس فيها.',
 'إذا تم إلقاء الفارس من جواده وتم الإمساك بقدمه في الأرجل، فإنه قد يسحب إذا نشر الحصان. لتقليل هذا المخاطر، يمكن اتخاذ عدد من الاحتياطات الأمنية.',
 'أولاً، يرتدي معظم الفرسان أحذية الفروسية بكعب ونعل ضيق وناعم.',
 'تتضمن صناعة الخزف تشكيل الطين إلى أشكال مختلفة ثم حرقها في فرن عالي الحرارة لصنع الأواني والأطباق والزهور والحلي.',
 'يتم إنتاج العديد من المنتجات من القماش المصنوع من القطن، بما في ذلك الملابس والبطانيات والمفروشات والمناشف والعديد من الأشياء الأخرى التي نستخدمها يوميًا.',
 'العديد من العلماء والمخترعين العرب ساهموا في تطوير العديد من التقنيات والمعرفة في العلوم الرياضية والفلك والطب والفلسفة والأدب.',


In [ ]:
result2 = translate(english[745:750])

In [ ]:
len(result)

10

In [ ]:
#translations.extend([(470+i, result[i]) for i in range(len(result))])

In [ ]:
translations

In [ ]:
df = pd.DataFrame({"translation":pd.Series(translations)})

In [ ]:
df

,translation
0,"(0, ""لدينا الآن فئران بعمر 4 أشهر غير مصابة با..."
1,"(1, حذر الدكتور إيهود أور ، أستاذ الطب في جامع..."
2,"(2, مثل بعض الخبراء الآخرين ، فهو متشكك من مدى..."
3,"(3, في يوم الاثنين، أعلنت سارة دانيوس، الأمين ..."
4,"(4, قالت دانيوس: ""في الوقت الحالي لا نفعل شيئً..."
...,...
896,"(906, نظرًا لأن المناطق قليلة السكان ، ولا يوج..."
897,"(907, ثقافة العمل اليابانية هي أكثر تسلسلًا ور..."
898,"(908, البدلات هي ملابس الأعمال القياسية ، ويدع..."
899,"(909, الانسجام في مكان العمل أمر حاسم ، حيث يؤ..."


In [ ]:

df.to_excel("translations_chatgpt_flores_eng_ar_0_910.xlsx")